In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

import darts
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller
from darts.models import DLinearModel

c:\Users\sonng\anaconda3\envs\tsff_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_df = pd.read_csv('../../data/electricity/train_df.csv')
test_df = pd.read_csv('../../data/electricity/test_df.csv')
X_train_df = pd.read_csv('../../data/electricity/y_train_df.csv')
X_test_df = pd.read_csv('../../data/electricity/X_test_df.csv')
y_train_df = pd.read_csv('../../data/electricity/y_train_df.csv')
y_test_df = pd.read_csv('../../data/electricity/y_test_df.csv')

In [4]:
value_cols = train_df.drop(columns=['datetime_utc']).columns

In [5]:
train_df['datetime_utc'] = pd.to_datetime(train_df['datetime_utc'])
train_df['datetime_utc'] = train_df['datetime_utc'].dt.tz_localize(None)


In [6]:
train_df

,datetime_utc,price_de_lag_336,price_de_avg_24,price_at_lag_24,price_at_avg_24,price_fr_lag_24,price_fr_avg_24,load_de_lag_24,load_de_lag_168,load_at,gen_de,gen_de_lag_168,gen_at,gen_fr,windon_de,solar_de,price_de
0,2022-10-31 23:00:00,135.71,143.192083,130.59,145.302083,136.77,142.511250,48193.0200,49161.9775,4868.0,49417.49,50407.76,4079.2,40284.0,16063.7425,0.0,80.65
1,2022-11-01 00:00:00,140.35,141.425000,119.09,144.504167,119.09,140.804167,44770.8675,48136.9350,4378.0,49446.61,49715.33,4027.1,39246.0,16839.0400,0.0,76.68
2,2022-11-01 01:00:00,139.95,138.880833,116.32,143.657500,116.32,138.950417,43981.9450,47600.7900,4829.0,48568.35,49051.50,3933.9,39366.5,17616.8775,0.0,55.26
3,2022-11-01 02:00:00,150.30,135.647500,114.60,142.480000,114.60,136.477083,43422.0775,47753.3075,4773.0,48487.32,49083.27,3766.6,37003.5,17984.3850,0.0,37.00
4,2022-11-01 03:00:00,149.51,132.467917,115.07,141.435417,115.07,134.017500,43581.7450,49360.8175,4833.0,49072.56,50384.18,3585.8,34949.5,18902.0375,0.0,38.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8993,2023-11-10 18:00:00,120.89,90.798333,112.41,95.982917,112.22,90.937500,64363.1600,62435.3950,7798.0,54701.73,60625.63,7268.6,63410.5,15818.6625,0.0,123.58
8994,2023-11-10 19:00:00,100.05,91.503333,99.15,96.557083,103.63,91.325000,62330.2025,59991.6850,7289.0,51345.73,60074.36,6770.3,61559.0,15185.0900,0.0,112.93
8995,2023-11-10 20:00:00,96.84,92.433333,85.54,97.372083,92.50,91.850000,58638.3150,56167.5975,6780.0,47738.81,58340.22,6173.9,59780.5,14189.9350,0.0,105.10
8996,2023-11-10 21:00:00,93.70,93.263750,84.04,98.202500,84.04,92.680417,55156.1275,52589.0625,6568.0,45213.73,56945.46,5898.0,58930.5,13123.5375,0.0,103.97


In [7]:
series = TimeSeries.from_dataframe(train_df, time_col='datetime_utc', value_cols=value_cols, freq='H')
train, val = series.split_before(pd.Timestamp('2023-10-09 23:00:00'))

In [8]:
scaler = Scaler()
series_scaled = scaler.fit_transform(train)
# series_scaled = scaler.transform(series)
# train_scaled, val_scaled = series_scaled.split_before(pd.Timestamp('2023-10-09 23:00:00'))

In [9]:
train_scaled, val_scaled = series_scaled.split_before(0.95)

In [10]:
train_scaled

<TimeSeries (DataArray) (datetime_utc: 7819, component: 16, sample: 1)>
array([[[0.54566914],
        [0.39011448],
        [0.54127432],
        ...,
        [0.36925814],
        [0.        ],
        [0.49840774]],

       [[0.54965193],
        [0.38661628],
        [0.53140316],
        ...,
        [0.38731282],
        [0.        ],
        [0.49500004]],

       [[0.54930859],
        [0.38157974],
        [0.5290255 ],
        ...,
...
        ...,
        [0.23736035],
        [0.15149105],
        [0.52102557]],

       [[0.57666458],
        [0.32254345],
        [0.57531695],
        ...,
        [0.20209067],
        [0.04570969],
        [0.54630432]],

       [[0.62660406],
        [0.31309724],
        [0.64803736],
        ...,
        [0.20182118],
        [0.00196726],
        [0.56643291]]])
Coordinates:
  * datetime_utc  (datetime_utc) datetime64[ns] 2022-10-31T23:00:00 ... 2023-...
  * component     (component) object 'price_de_lag_336' ... 'price_de'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [11]:
val_scaled

<TimeSeries (DataArray) (datetime_utc: 413, component: 16, sample: 1)>
array([[[0.5970335 ],
        [0.31133619],
        [0.55300813],
        ...,
        [0.22211692],
        [0.        ],
        [0.54372924]],

       [[0.54255328],
        [0.31131886],
        [0.52772079],
        ...,
        [0.24369738],
        [0.        ],
        [0.52754054]],

       [[0.52697402],
        [0.31179975],
        [0.51510287],
        ...,
...
        ...,
        [0.12896544],
        [0.        ],
        [0.53218427]],

       [[0.52374658],
        [0.33630527],
        [0.51187543],
        ...,
        [0.11265179],
        [0.        ],
        [0.52591823]],

       [[0.52717144],
        [0.33736851],
        [0.5073862 ],
        ...,
        [0.13757771],
        [0.        ],
        [0.51845049]]])
Coordinates:
  * datetime_utc  (datetime_utc) datetime64[ns] 2023-09-22T18:00:00 ... 2023-...
  * component     (component) object 'price_de_lag_336' ... 'price_de'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [12]:
# model = DLinearModel(input_chunk_length=24*2, output_chunk_length=24, n_epochs=6, batch_size = 128, random_state=42, use_static_covariates=False, shared_weights=True)
# model.fit(train_scaled, val_series=val_scaled)
# 8.714575167647306

In [13]:
def smape_loss(y_true, y_pred):
    """
    Calculate the Symmetric Mean Absolute Percentage Error (SMAPE).

    Args:
    y_true (array): True values.
    y_pred (array): Predicted values.

    Returns:
    float: SMAPE score.
    """
    # Avoid division by zero by adding a small epsilon
    epsilon = np.finfo(np.float64).eps
    denominator = np.maximum(np.abs(y_true) + np.abs(y_pred) + epsilon, 0.5 + epsilon)

    # Calculate SMAPE
    smape_value = 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / denominator)
    return smape_value


In [95]:
model = DLinearModel(input_chunk_length=24*2, output_chunk_length=24, n_epochs=5, batch_size = 32, random_state=658, use_static_covariates=False, shared_weights=True)
model.fit(train_scaled, val_series=val_scaled)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type             | Params
-----------------------------------------------------
0 | criterion       | MSELoss          | 0     
1 | train_metrics   | MetricCollection | 0     
2 | val_metrics     | MetricCollection | 0     
3 | decomposition   | _SeriesDecomp    | 0     
4 | linear_seasonal | Linear           | 1.2 K 
5 | linear_trend    | Linear           | 1.2 K 
-----------------------------------------------------
2.4 K     Trainable params
0         Non-trainable params
2.4 K     Total params
0.009     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Epoch 4: 100%|██████████| 243/243 [00:02<00:00, 82.07it/s, train_loss=0.00464, val_loss=0.00719]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 243/243 [00:02<00:00, 82.01it/s, train_loss=0.00464, val_loss=0.00719]


DLinearModel(shared_weights=True, kernel_size=25, const_init=True, use_static_covariates=False, input_chunk_length=48, output_chunk_length=24, n_epochs=5, batch_size=32, random_state=658)

In [96]:
prediction = model.predict(n=24, series=val_scaled)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 100.08it/s]


In [97]:
pred = prediction.all_values()[:,-1]
pred

array([[0.51557466],
       [0.5103645 ],
       [0.50347747],
       [0.50147478],
       [0.50543605],
       [0.513484  ],
       [0.51626449],
       [0.52038847],
       [0.50984944],
       [0.50209767],
       [0.49749422],
       [0.49015215],
       [0.48840703],
       [0.4881625 ],
       [0.49402309],
       [0.50617227],
       [0.52441806],
       [0.5430474 ],
       [0.54927405],
       [0.54235476],
       [0.52098444],
       [0.50270191],
       [0.49079575],
       [0.48229176]])

In [98]:
pred = scaler.inverse_transform(prediction).all_values()[:,-1]
pred

array([[100.64963142],
       [ 94.57974591],
       [ 86.55628676],
       [ 84.22313734],
       [ 88.8380545 ],
       [ 98.21399229],
       [101.45328929],
       [106.25777358],
       [ 93.97969737],
       [ 84.94880618],
       [ 79.58573694],
       [ 71.03216089],
       [ 68.99907715],
       [ 68.71419439],
       [ 75.54183546],
       [ 89.69575659],
       [110.95227911],
       [132.65565088],
       [139.90976077],
       [131.84871562],
       [106.95208473],
       [ 85.65275202],
       [ 71.78195859],
       [ 61.87472761]])

In [99]:
test_df['price_de']

0      87.30
1      85.89
2      85.88
3      83.21
4      82.71
5      85.44
6      86.30
7      88.00
8      92.00
9      94.89
10     91.28
11     90.22
12     87.94
13     87.80
14     91.50
15    102.87
16    115.72
17    127.98
18    135.76
19    124.47
20    113.51
21    102.58
22     92.38
23     87.41
Name: price_de, dtype: float64